In [1]:
# Jupyter Notebook 01 — Load + Flatten (MX Pilot Top10 SKUs)
# ---------------------------------------------------------
# Objetivo:
# 1) Cargar tu JSON (3 fábricas: Toluca, Irapuato, Valle de México)
# 2) Aplanarlo a DataFrames:
#    - df_factories (1 fila por fábrica)
#    - df_top10 (1 fila por SKU del top_10 de cada fábrica)
# 3) Guardar outputs como CSV (y opcional Parquet) para siguientes notebooks

import json
from pathlib import Path
import pandas as pd
import numpy as np


In [4]:
# === CONFIG ===
# Ajusta la ruta a donde guardaste el JSON
JSON_PATH = Path("mx_pilot_top10_skus.json")

assert JSON_PATH.exists(), f"No encuentro el archivo: {JSON_PATH.resolve()}"
JSON_PATH


PosixPath('mx_pilot_top10_skus.json')

In [5]:
# === LOAD JSON ===
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# sanity checks rápidos
for k in ["dataset_version", "as_of_date", "factories"]:
    assert k in data, f"Falta la llave '{k}' en el JSON."

len(data["factories"]), data["dataset_version"], data["as_of_date"]


(3, 'mx_pilot_sku_v1', '2026-01-14')

In [6]:
# === HELPERS ===

def _safe_get(dct, path, default=None):
    """
    path: lista de llaves, ej. ["geo","lat"]
    """
    cur = dct
    for p in path:
        if not isinstance(cur, dict) or p not in cur:
            return default
        cur = cur[p]
    return cur

def flatten_factories(data: dict) -> pd.DataFrame:
    rows = []
    for fac in data["factories"]:
        rows.append({
            "dataset_version": data.get("dataset_version"),
            "as_of_date": data.get("as_of_date"),
            "factory_id": fac.get("factory_id"),
            "factory_name": fac.get("factory_name"),
            "state": fac.get("state"),
            "city": fac.get("city"),
            "lat": _safe_get(fac, ["geo", "lat"]),
            "lon": _safe_get(fac, ["geo", "lon"]),
            "overall_risk_oos_7d": fac.get("overall_risk_oos_7d"),
            "critical_sku_id": _safe_get(fac, ["critical_sku", "sku_id"]),
            "critical_sku_name": _safe_get(fac, ["critical_sku", "sku_name"]),
            "top10_count": len(fac.get("top_10_skus", []))
        })
    return pd.DataFrame(rows)

def flatten_top10_skus(data: dict) -> pd.DataFrame:
    rows = []
    for fac in data["factories"]:
        factory_id = fac.get("factory_id")
        factory_name = fac.get("factory_name")
        state = fac.get("state")
        city = fac.get("city")
        lat = _safe_get(fac, ["geo", "lat"])
        lon = _safe_get(fac, ["geo", "lon"])
        overall_risk = fac.get("overall_risk_oos_7d")

        for rank, sku in enumerate(fac.get("top_10_skus", []), start=1):
            cur = sku.get("current_state", {}) or {}
            der = sku.get("derived", {}) or {}

            # Drivers: los aplanamos a 3 columnas principales (top3) y guardamos el resto como json string
            drivers = sku.get("drivers", []) or []
            drivers_sorted = drivers[:]
            # si vienen sin orden, ordena por impacto desc
            if len(drivers_sorted) > 1 and all("impact" in x for x in drivers_sorted):
                drivers_sorted = sorted(drivers_sorted, key=lambda x: x.get("impact", 0), reverse=True)

            def drv(i, field):
                if i < len(drivers_sorted):
                    return drivers_sorted[i].get(field)
                return None

            rows.append({
                "dataset_version": data.get("dataset_version"),
                "as_of_date": data.get("as_of_date"),

                "factory_id": factory_id,
                "factory_name": factory_name,
                "state": state,
                "city": city,
                "lat": lat,
                "lon": lon,
                "factory_overall_risk_oos_7d": overall_risk,

                "rank_in_factory": rank,

                "sku_id": sku.get("sku_id"),
                "sku_name": sku.get("sku_name"),
                "sku_family": sku.get("sku_family"),

                "risk_oos_7d": sku.get("risk_oos_7d"),
                "confidence": sku.get("confidence"),

                "days_of_supply": der.get("days_of_supply"),
                "stockout_date_est": der.get("stockout_date_est"),
                "stock_needed_next_7d": der.get("stock_needed_next_7d"),

                # Core current_state features
                "on_hand_units": cur.get("on_hand_units"),
                "in_transit_units": cur.get("in_transit_units"),
                "on_order_units": cur.get("on_order_units"),
                "daily_demand_units": cur.get("daily_demand_units"),
                "lead_time_days_mean": cur.get("lead_time_days_mean"),
                "lead_time_days_std": cur.get("lead_time_days_std"),
                "promo_uplift_index": cur.get("promo_uplift_index"),
                "scrap_rate_7d": cur.get("scrap_rate_7d"),

                # Top drivers flattened
                "driver1_feature": drv(0, "feature"),
                "driver1_impact": drv(0, "impact"),
                "driver2_feature": drv(1, "feature"),
                "driver2_impact": drv(1, "impact"),
                "driver3_feature": drv(2, "feature"),
                "driver3_impact": drv(2, "impact"),

                # Keep raw drivers for later
                "drivers_json": json.dumps(drivers, ensure_ascii=False)
            })

    df = pd.DataFrame(rows)

    # Tip: asegurar tipos numéricos
    num_cols = [
        "factory_overall_risk_oos_7d", "risk_oos_7d", "confidence",
        "days_of_supply", "stock_needed_next_7d",
        "on_hand_units", "in_transit_units", "on_order_units", "daily_demand_units",
        "lead_time_days_mean", "lead_time_days_std", "promo_uplift_index", "scrap_rate_7d",
        "driver1_impact", "driver2_impact", "driver3_impact",
        "lat", "lon"
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    return df


In [7]:
# === BUILD DATAFRAMES ===
df_factories = flatten_factories(data)
df_top10 = flatten_top10_skus(data)

df_factories, df_top10.shape


(   dataset_version  as_of_date factory_id            factory_name  \
 0  mx_pilot_sku_v1  2026-01-14  MX_TOL_01           Planta Toluca   
 1  mx_pilot_sku_v1  2026-01-14  MX_IRA_02         Planta Irapuato   
 2  mx_pilot_sku_v1  2026-01-14  MX_VDM_03  Planta Valle de México   
 
               state                city      lat       lon  \
 0  Estado de México              Toluca  19.2826  -99.6557   
 1        Guanajuato            Irapuato  20.6767 -101.3563   
 2  Estado de México  Cuautitlán Izcalli  19.6469  -99.2460   
 
    overall_risk_oos_7d critical_sku_id            critical_sku_name  \
 0                 0.82     TOL-INF-004  Infant Formula Stage 1 800g   
 1                 0.46     IRA-DAI-006         Yogurt Strawberry 1L   
 2                 0.68     VDM-WAT-003          Water 600ml 24-pack   
 
    top10_count  
 0            2  
 1            1  
 2            1  ,
 (4, 33))

In [8]:
# === QUICK VIEW ===
df_factories


,dataset_version,as_of_date,factory_id,factory_name,state,city,lat,lon,overall_risk_oos_7d,critical_sku_id,critical_sku_name,top10_count
0,mx_pilot_sku_v1,2026-01-14,MX_TOL_01,Planta Toluca,Estado de México,Toluca,19.2826,-99.6557,0.82,TOL-INF-004,Infant Formula Stage 1 800g,2
1,mx_pilot_sku_v1,2026-01-14,MX_IRA_02,Planta Irapuato,Guanajuato,Irapuato,20.6767,-101.3563,0.46,IRA-DAI-006,Yogurt Strawberry 1L,1
2,mx_pilot_sku_v1,2026-01-14,MX_VDM_03,Planta Valle de México,Estado de México,Cuautitlán Izcalli,19.6469,-99.2460,0.68,VDM-WAT-003,Water 600ml 24-pack,1


In [9]:
# Top 10 SKUs (vista rápida)
df_top10.sort_values(["factory_id", "rank_in_factory"]).head(15)


,dataset_version,as_of_date,factory_id,factory_name,state,city,lat,lon,factory_overall_risk_oos_7d,rank_in_factory,...,lead_time_days_std,promo_uplift_index,scrap_rate_7d,driver1_feature,driver1_impact,driver2_feature,driver2_impact,driver3_feature,driver3_impact,drivers_json
2,mx_pilot_sku_v1,2026-01-14,MX_IRA_02,Planta Irapuato,Guanajuato,Irapuato,20.6767,-101.3563,0.46,1,...,1.1,1.05,0.017,days_of_supply,0.16,promo_uplift_index,0.10,None,NaN,"[{""feature"": ""days_of_supply"", ""impact"": 0.16}..."
0,mx_pilot_sku_v1,2026-01-14,MX_TOL_01,Planta Toluca,Estado de México,Toluca,19.2826,-99.6557,0.82,1,...,2.0,1.10,0.006,days_of_supply,0.32,lead_time_days_std,0.22,promo_uplift_index,0.12,"[{""feature"": ""days_of_supply"", ""impact"": 0.32}..."
1,mx_pilot_sku_v1,2026-01-14,MX_TOL_01,Planta Toluca,Estado de México,Toluca,19.2826,-99.6557,0.82,2,...,1.4,1.12,0.019,days_of_supply,0.21,promo_uplift_index,0.18,None,NaN,"[{""feature"": ""promo_uplift_index"", ""impact"": 0..."
3,mx_pilot_sku_v1,2026-01-14,MX_VDM_03,Planta Valle de México,Estado de México,Cuautitlán Izcalli,19.6469,-99.2460,0.68,1,...,1.5,1.07,0.004,days_of_supply,0.24,lead_time_days_std,0.18,None,NaN,"[{""feature"": ""days_of_supply"", ""impact"": 0.24}..."


In [10]:
# === VALIDATIONS (importantes para que la UI y el modelado no fallen) ===

# 1) cada fábrica debe tener lat/lon
assert df_factories["lat"].notna().all() and df_factories["lon"].notna().all(), "Hay fábricas sin lat/lon."

# 2) risk entre 0 y 1
bad_risk = df_top10[(df_top10["risk_oos_7d"] < 0) | (df_top10["risk_oos_7d"] > 1)]
assert bad_risk.empty, f"Hay risks fuera de [0,1]:\n{bad_risk[['factory_id','sku_id','risk_oos_7d']].head()}"

# 3) top10 ordenado desc por factory (si no, lo ordenamos)
def is_sorted_desc(group, col):
    vals = group[col].values
    return np.all(vals[:-1] >= vals[1:])

unsorted = []
for fid, g in df_top10.groupby("factory_id"):
    g2 = g.sort_values("rank_in_factory")
    if not is_sorted_desc(g2, "risk_oos_7d"):
        unsorted.append(fid)

unsorted


[]

In [11]:
# Si detectamos fábricas con top10 no ordenado por risk, re-rankeamos automáticamente (no rompe nada)
if unsorted:
    df_top10 = (
        df_top10.sort_values(["factory_id", "risk_oos_7d"], ascending=[True, False])
               .groupby("factory_id", as_index=False, group_keys=False)
               .apply(lambda g: g.assign(rank_in_factory=np.arange(1, len(g)+1)))
               .sort_values(["factory_id", "rank_in_factory"])
    )

# actualizar críticos por fábrica usando max risk real de df_top10
crit = (
    df_top10.sort_values(["factory_id", "risk_oos_7d"], ascending=[True, False])
           .groupby("factory_id", as_index=False)
           .first()[["factory_id", "sku_id", "sku_name", "risk_oos_7d"]]
           .rename(columns={"sku_id":"critical_sku_id_calc", "sku_name":"critical_sku_name_calc", "risk_oos_7d":"critical_risk_calc"})
)

df_factories = df_factories.merge(crit, on="factory_id", how="left")
df_factories


,dataset_version,as_of_date,factory_id,factory_name,state,city,lat,lon,overall_risk_oos_7d,critical_sku_id,critical_sku_name,top10_count,critical_sku_id_calc,critical_sku_name_calc,critical_risk_calc
0,mx_pilot_sku_v1,2026-01-14,MX_TOL_01,Planta Toluca,Estado de México,Toluca,19.2826,-99.6557,0.82,TOL-INF-004,Infant Formula Stage 1 800g,2,TOL-INF-004,Infant Formula Stage 1 800g,0.82
1,mx_pilot_sku_v1,2026-01-14,MX_IRA_02,Planta Irapuato,Guanajuato,Irapuato,20.6767,-101.3563,0.46,IRA-DAI-006,Yogurt Strawberry 1L,1,IRA-DAI-006,Yogurt Strawberry 1L,0.46
2,mx_pilot_sku_v1,2026-01-14,MX_VDM_03,Planta Valle de México,Estado de México,Cuautitlán Izcalli,19.6469,-99.2460,0.68,VDM-WAT-003,Water 600ml 24-pack,1,VDM-WAT-003,Water 600ml 24-pack,0.68


In [12]:
# === OPTIONAL: construir df_skus (full list) ===
# Nota: tu JSON actual trae solo top_10_skus; este df_skus será igual a df_top10 por ahora.
# En el futuro, cuando tengas "skus" (30 por fábrica), aquí lo aplanamos igual.

df_skus = df_top10.copy()
df_skus.shape


(4, 33)

In [13]:
# === SAVE OUTPUTS ===
OUT_DIR = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

factories_csv = OUT_DIR / "factories.csv"
top10_csv = OUT_DIR / "top10_skus.csv"

df_factories.to_csv(factories_csv, index=False)
df_top10.to_csv(top10_csv, index=False)

factories_csv, top10_csv


(PosixPath('data/processed/factories.csv'),
 PosixPath('data/processed/top10_skus.csv'))

In [14]:
# (Opcional) Guardar parquet si tienes pyarrow instalado (suele estar)
try:
    factories_parquet = OUT_DIR / "factories.parquet"
    top10_parquet = OUT_DIR / "top10_skus.parquet"
    df_factories.to_parquet(factories_parquet, index=False)
    df_top10.to_parquet(top10_parquet, index=False)
    factories_parquet, top10_parquet
except Exception as e:
    print("Parquet no se guardó (no pasa nada). Error:", e)


In [15]:
# === SUMMARY PARA EL SIGUIENTE NOTEBOOK ===
# - df_factories: mapa + semáforo + SKU crítico
# - df_top10: tabla Top 10 por fábrica + features
print("df_factories rows:", len(df_factories))
print("df_top10 rows:", len(df_top10))
print("Factories:", df_factories["factory_name"].tolist())


df_factories rows: 3
df_top10 rows: 4
Factories: ['Planta Toluca', 'Planta Irapuato', 'Planta Valle de México']
